In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
path ='/kaggle/input/whats-cooking-kernels-only/'

with open(path + 'train.json','r') as f:
    train_data = json.load(f)
with open(path + 'test.json','r') as f:
    test_data = json.load(f)

In [ ]:
train_data=pd.DataFrame(train_data)

In [ ]:
test_data=pd.DataFrame(test_data)

In [ ]:
train_data.head()

In [ ]:
#Data Exploration
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data['cuisine'].unique()

In [ ]:
import random
def random_col(num_col):
    colors = []
    for i in range(num_col):        
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

In [ ]:
train_data['cuisine'].value_counts().plot.bar(color=random_col(20),figsize=(16,6))

In [ ]:
##most commonly used ingredients.
from collections import Counter
most_commoning = Counter([item for lists in train_data['ingredients'] for item in lists])
print(most_commoning.most_common(20))

In [ ]:
##plotting a graph on most common ingredients.
import matplotlib.pyplot as plt
import seaborn as sns
ing_df = pd.DataFrame(most_commoning.most_common(20))
ing_df.columns = ['ingredients','tot_count']
plt.figure(figsize=(7,9))
sns.barplot(x='tot_count',y='ingredients',data=ing_df)

In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
stop_words = set(stopwords.words('english'))
stemer = PorterStemmer()

In [ ]:
import nltk
import string
punctuations = string.punctuation
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import re
#import unicodedata
lmz = WordNetLemmatizer()
 
def clean(text):
 
 
 txt = str(text).lower()
 txt = "".join(c for c in txt if c not in punctuations) 
 words = txt.split() 
 words = [i for i in words if i.isalpha()] 
 pattern = '[0-9]'
 words = [re.sub(pattern, '', i) for i in words]
 words = [re.sub(r'[^a-zA-Z]+$', '', i) for i in words] 
 
 clean_txt = " ".join(words) 
 return clean_txt

In [ ]:
##using stemming in preprocessing
def clean_stem(text):
 
 
 txt = str(text).lower()
 txt = "".join(c for c in txt if c not in punctuations) 
 words = txt.split() 
 words = [i for i in words if i.isalpha()] 
 pattern = '[0-9]'
 words = [re.sub(pattern, '', i) for i in words]
 words = [re.sub(r'[^a-zA-Z]+$', '', i) for i in words] 
 words = " ".join(words) 
 words=[stemer.stem(word) for word in words.split(" ") if words not in stop_words]
 
 clean_txt = " ".join(words) 
 return clean_txt



In [ ]:
#Applying the cleaning function to both test and train datasets
train_data['clean_text'] = train_data['ingredients'].apply(lambda x: clean(x))
train_data['cleanstem_text']=train_data['ingredients'].apply(lambda x:clean_stem(x))
test_data['clean_text'] = test_data['ingredients'].apply(lambda x: clean(x))

In [ ]:
train_data.head()

In [ ]:
cvec = CountVectorizer()
train_cv = cvec.fit_transform(train_data['clean_text'])
test_cv = cvec.transform(test_data['clean_text'])
print(train_cv[0].todense())

In [ ]:
tfidf = TfidfVectorizer(min_df = 2,max_df = 0.5,ngram_range = (1,2))
train_tfidf = tfidf.fit_transform(train_data['clean_text'])
test_tfidf = tfidf.transform(test_data['clean_text'])

In [ ]:
#label encode the target data
le = LabelEncoder()
train_data['cat_cuisine']=le.fit_transform(train_data['cuisine'])

In [ ]:
#to check the mappings of the target class to its integer value
le_name_map = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_map)


In [ ]:
y_data=le.fit_transform(train_data['cuisine'])

In [ ]:

x_traintf, x_validtf, y_traintf, y_validtf = train_test_split(train_tfidf ,y_data , test_size=0.2, random_state=42)
x_traincv,x_validcv,y_traincv,y_validcv=train_test_split(train_cv ,y_data , test_size=0.2, random_state=42)

In [ ]:
train_data['cat_cuisine'].value_counts()

In [ ]:
mnb = MultinomialNB(alpha = 2.0)
metrics_cv = model_selection.cross_val_score(mnb,train_cv,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score:",metrics_cv)
metric_tfidf = model_selection.cross_val_score(mnb,train_tfidf,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of tfidf:",metric_tfidf)

In [ ]:
mnb.fit(train_cv,train_data['cat_cuisine'])

In [ ]:
y_mnbpreds=mnb.predict(test_cv)

In [ ]:
y_mnbpreds

In [ ]:
y_mnbpreds=le.inverse_transform(y_mnbpreds)

In [ ]:
y_mnbpreds

In [ ]:
test_data.head()

In [ ]:
##will try to balance the dataset and apply the model on balanced dataset
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
#undersampling on train countvectirizer data
undersample =RandomUnderSampler(sampling_strategy='majority')
x_undercv,y_undercv=undersample.fit_resample(train_cv,train_data['cat_cuisine'])

In [ ]:
#undersampling on train tfidf data
x_undertfidf,y_undertfidf=undersample.fit_resample(train_tfidf,train_data['cat_cuisine'])

In [ ]:
#trying multinomial Naive bayes on balanced dataset and without balanced dataset
metrics_cv = model_selection.cross_val_score(mnb,train_cv,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("mean_score cv:",metrics_cv.mean())
metric_tfidf = model_selection.cross_val_score(mnb,train_tfidf,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of tfidf:",metric_tfidf.mean())
metrics_cvbal = model_selection.cross_val_score(mnb,x_undercv,y_undercv,cv = 10,scoring = 'accuracy')
print("score of cv balanced:",metrics_cvbal.mean())
metric_tfidfbal = model_selection.cross_val_score(mnb,x_undertfidf,y_undertfidf,cv = 10,scoring = 'accuracy')
print("score of tfidf balanced:",metric_tfidfbal.mean())

In [ ]:
#mnb.fit(x_undercv,y_undercv)
##since we got high accuracy with CV train test data, therefore we are going with count vectorizer dataset
#y_mnbpredsbal=mnb.predict(test_cv)
#y_mnbpredsbal=le.inverse_transform(y_mnbpredsbal)
#submission = pd.DataFrame({'Id':test_data['id'],'cuisine':y_mnbpredsbal})
#submission.to_csv('submission.csv',index=False)

In [ ]:
##we are now aware that undersamping did not give much accuracy therefore, we can try without sampling the dataset
#now will go for others models, without sampling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import model_selection

In [ ]:
logreg=LogisticRegression(random_state=0)
metrics_cv = model_selection.cross_val_score(logreg,train_cv,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of cv:",metrics_cv.mean())
metric_tfidf = model_selection.cross_val_score(logreg,train_tfidf,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of tfidf:",metric_tfidf.mean())
metrics_cvbal = model_selection.cross_val_score(logreg,x_undercv,y_undercv,cv = 10,scoring = 'accuracy')
print("score of cv balanced:",metrics_cvbal.mean())
metric_tfidfbal = model_selection.cross_val_score(logreg,x_undertfidf,y_undertfidf,cv = 10,scoring = 'accuracy')
print("score of tfidf balanced:",metric_tfidfbal.mean())


In [ ]:
#using Support vector machine
lsvc=LinearSVC()
metrics_cv = model_selection.cross_val_score(lsvc,train_cv,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of cv:",metrics_cv.mean())
metric_tfidf = model_selection.cross_val_score(lsvc,train_tfidf,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of tfidf:",metric_tfidf.mean())
metrics_cvbal = model_selection.cross_val_score(lsvc,x_undercv,y_undercv,cv = 10,scoring = 'accuracy')
print("score of cv balanced:",metrics_cvbal.mean())
metric_tfidfbal = model_selection.cross_val_score(lsvc,x_undertfidf,y_undertfidf,cv = 10,scoring = 'accuracy')
print("score of tfidf balanced:",metric_tfidfbal.mean())

In [ ]:
#metrics_cv = model_selection.cross_val_score(lsvc,train_cv,train_data['cat_cuisine'],cv = 10,scoring = 'f1_macro')
#print("score of cv:",metrics_cv.mean())

In [ ]:
#Using KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
metrics_cv = model_selection.cross_val_score(knn,train_cv,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of cv:",metrics_cv.mean())
metric_tfidf = model_selection.cross_val_score(knn,train_tfidf,train_data['cat_cuisine'],cv = 10,scoring = 'accuracy')
print("score of tfidf:",metric_tfidf.mean())
metrics_cvbal = model_selection.cross_val_score(knn,x_undercv,y_undercv,cv = 10,scoring = 'accuracy')
print("score of cv balanced:",metrics_cvbal.mean())
metric_tfidfbal = model_selection.cross_val_score(knn,x_undertfidf,y_undertfidf,cv = 10,scoring = 'accuracy')
print("score of tfidf balanced:",metric_tfidfbal.mean())

In [ ]:
#logreg.fit(train_cv,train_data['cat_cuisine'])

In [ ]:
#y_predlogregcv=logreg.predict(test_cv)

In [ ]:
#y_predlogregcv=le.inverse_transform(y_predlogregcv)

In [ ]:
lsvc.fit(train_tfidf,train_data['cat_cuisine'])
y_predsvctf=lsvc.predict(test_tfidf)
y_predsvctf=le.inverse_transform(y_predsvctf)

In [ ]:
submission = pd.DataFrame({'Id':test_data['id'],'cuisine':y_predsvctf})
submission.to_csv('submission.csv',index=False)

In [ ]:
## now trying to evaluate the model on train and valid data which we obtained using train and test
#FOR Naive Bayes
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.metrics import confusion_matrix
mnb.fit(x_traintf,y_traintf)
y_predmnbtf=mnb.predict(x_validtf)
mnb.fit(x_traincv,y_traincv)
y_predmnbcv=mnb.predict(x_validcv)

In [ ]:
predmnb_tf=classification_report(y_validtf, y_predmnbtf)
predmnb_cv=classification_report(y_validcv, y_predmnbcv)
print("report of Naivebyes with Tfidf:\n",predmnb_tf)
print("report of Naivebyes with Count Vec:\n",predmnb_cv)

In [ ]:
#FOR Logistic reg
logreg.fit(x_traintf,y_traintf)
y_predlogregtf=logreg.predict(x_validtf)
logreg.fit(x_traincv,y_traincv)
y_predlogregcv=logreg.predict(x_validcv)
predlogreg_tf=classification_report(y_validtf, y_predlogregtf)
predlogreg_cv=classification_report(y_validcv, y_predlogregcv)
print("report of Logistic Reg with Tfidf:\n",predlogreg_tf)
print("report of Logistic Reg with Count Vec:\n",predlogreg_cv)

In [ ]:
#FOR Linear SVC
lsvc.fit(x_traintf,y_traintf)
y_predlsvctf=lsvc.predict(x_validtf)
lsvc.fit(x_traincv,y_traincv)
y_predlsvccv=lsvc.predict(x_validcv)
predlsvc_tf=classification_report(y_validtf, y_predlsvctf)
predlsvc_cv=classification_report(y_validcv, y_predlsvccv)
print("report of Linear SVC with Tfidf:\n",predlsvc_tf)
print("report of  Linear SVC with Count Vec:\n",predlsvc_cv)

In [ ]:
#FOR Linear knn
knn.fit(x_traintf,y_traintf)
y_predknntf=knn.predict(x_validtf)
knn.fit(x_traincv,y_traincv)
y_predknncv=knn.predict(x_validcv)
predknn_tf=classification_report(y_validtf, y_predknntf)
predknn_cv=classification_report(y_validcv, y_predknncv)
print("report of knn with Tfidf:\n",predknn_tf)
print("report of  knn with Count Vec:\n",predknn_cv)